In [3]:
# remove augmented files from MIDI_Files
import os
import glob

files = glob.glob('MIDI_Files/*_modified*.mid')

for file in files:
    try:
        os.remove(file)
    except Exception as e:
        print(f"Error removing file {file}: {e}")

In [2]:
import pretty_midi

def copy_instrument(original_instrument):
    new_instrument = pretty_midi.Instrument(program=original_instrument.program)

    for note in original_instrument.notes:
        new_instrument.notes.append(pretty_midi.Note(
            start=note.start,
            end=note.end,
            pitch=note.pitch,
            velocity=note.velocity
        ))

    return new_instrument

def add_imperfection(midi_data, pitch_offset=-4):
    # Crear una copia de los datos MIDI
    modified_midi_data = pretty_midi.PrettyMIDI()

    # Manipular el pitch de las notas
    for instrument in midi_data.instruments:
        new_instrument = copy_instrument(instrument)

        for note in new_instrument.notes:
            if int(note.pitch + pitch_offset) < 0 or int(note.pitch + pitch_offset) > 127:
                return None

            # Aplicar el offset de pitch
            note.pitch = int(note.pitch + pitch_offset)

        modified_midi_data.instruments.append(new_instrument)

    return modified_midi_data

def process_dataset(dataset_path):
    debug = False
    n_files_before = len(os.listdir(dataset_path))
    print("Number of files before augmentation: ", n_files_before)
    
    pitch_offset_list = [-16, -12, -8, -4, 4, 8, 12]
    for midi_file in os.listdir(dataset_path):
        if midi_file.endswith(".mid"):
            midi_file_path = os.path.join(dataset_path, midi_file)
    
            # Cargar el archivo MIDI original
            original_midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    
            # Iterar sobre los valores de pitch_offset
            for i, pitch_offset in enumerate(pitch_offset_list, start=1):
                # Modificar los datos MIDI
                modified_midi_data = add_imperfection(original_midi_data, pitch_offset=pitch_offset)
    
                # Si el resultado es None, omitir la escritura del archivo
                if modified_midi_data is None:
                    if debug:
                        print(f"Skipping {midi_file} modification_{i} due to invalid pitch after modification.")
                    continue
    
                # Guardar el archivo MIDI modificado
                modified_output_path = os.path.join(dataset_path, f"{midi_file.replace('.mid', f'_modified{i}.mid')}")
                try:
                    modified_midi_data.write(modified_output_path)
                except ValueError as e:
                    print(f"Error writing modified file {i} for {midi_file}: {e}")
            
    n_files_after = len(os.listdir(dataset_path))
    print("Number of files after augmentation: ", n_files_after)
    print("Number of failures during augmentation: ", n_files_after - (n_files_before * (len(pitch_offset_list)+1)))
        
def main():
    # Specify the dataset directory
    dataset_path = "MIDI_Files/"

    # Process the dataset
    process_dataset(dataset_path)

if __name__ == "__main__":
    main()


Number of files before augmentation:  61


c:\users\santi\documents\github\groove2groove\venv\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Number of files after augmentation:  488
Number of failures during augmentation:  0
